In [27]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [28]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [29]:


# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)



{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [30]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
print(Xtr[:4])
print(Ytr[:4])
print(Xtr.shape[0])
print(Xtr.shape[0])
print(Xtr.shape[1])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
tensor([[ 0,  0,  0],
        [ 0,  0, 25],
        [ 0, 25, 21],
        [25, 21,  8]])
tensor([25, 21,  8,  5])
182625
182625
3


In [31]:
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt-t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

    

In [38]:
n_embd = 10
n_hidden = 64
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd), generator=g)
W1 = torch.randn((n_embd* block_size, n_hidden), generator=g) * (5/3) / ((n_embd * block_size) ** 0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.1
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1
bngain = torch.randn((1, n_hidden), generator=g)* 1.0
bnbias = torch.randn((1, n_hidden), generator=g)* 0.1
parameters = [C, W1, W2, b1, b2,bngain, bnbias ]
for p in parameters:
  p.requires_grad = True





In [39]:
#We now construct a batch
batch_size = 32;
ix = torch.randint(0, Xtr.shape[0], (batch_size, ))
Xb,Yb = Xtr[ix], Ytr[ix]
n = batch_size



In [40]:
emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)
hprebn = embcat @W1 + b1
#Batch norm layer
bnmeani = 1/n * hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff **2
bnvar = 1 / (n - 1) * (bndiff2).sum(0, keepdim=True)
bnvar_inv=(bnvar + 1e-5) **-0.5
bnraw = bndiff* bnvar_inv
hpreact = bngain * bnraw + bnbias

#activation function
h = torch.tanh(hpreact)

#linear layer
logits = h@W2 + b2

#now we do crossentropy
#Shape of  logits 32 * 27
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes
#The reason we subtract max from all values is that in the next step we perform exp
#and since exp grow very fast if we  have a high positive no the exp will reach inf
#Hence we subtract max which will make the highest no 1
#since e^0 is 1 and remaing no will be neg
counts =norm_logits.exp()
counts_sum = counts.sum(1, keepdim=True)#We take  for each row  we wil ahve one sum
#and we will divide each entry of corresponding row with its corresponding row sum

counts_sum_inv = counts_sum**-1 #This is same as 1/counts_sum
probs = counts * counts_sum_inv
logprobs = probs.log();
loss = -logprobs[range(n), Yb].mean()

for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, 
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss


tensor(3.3990, grad_fn=<NegBackward0>)

In [41]:
# #First we calc derivative of logprobs
# we can see that the loss is defined as loss = -logprobs(range(n), Yb).mean()
# basically what this means is this:
# since logprobs is 32*27 for each of the 32 rows we select one element corresponding to correct output(Yb)
# Then we add them together and take their mean
# We can simplify this using simple example
# lets suppose ls = (a + b + c) / 3
# then dls/da =1/3 dls/db = 1/3 and dls/dc = 1/3
# Our logprobs obeys this same example
# so for all the elements that are in the loss i.e 32 elements their derivative will be 1/n
# but what about the remaining 32*27 - 32 elements?
# since they do not participate in the loss their derivative will intuitively be 0
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0/n
cmp('logprobs', dlogprobs, logprobs)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
